## Оглавление

<a href='#1'>1. Знакомство с данными </a>
* <a href='#1.1'>1.1 Предобработка данных </a>

<a href='#2'>2. Лемматизация</a>

<a href='#3'>3. Категоризация данных</a>

<a href='#4'>4. Анализ данных</a>
* <a href='#4.1'>4.1 Зависимость между наличием детей и возвратом кредита в срок </a>
* <a href='#4.2'>4.2 Зависимость между семейным положением и возвратом кредита в срок </a>
* <a href='#4.3'>4.3 Зависимость между уровнем дохода и возвратом кредита в срок </a>
* <a href='#4.4'>4.4 Зависимость цели кредита и его возвратом в срок </a>

<a href='#5'> Общий вывод</a>

<a id='1'></a>
## 1. Знакомство с данными

In [5]:
#Импорт библиотек

import pandas as pd #для описания и анализа данных
import matplotlib.pyplot as plt # для работы с графиками
#магия, чтобы график был сразу после выполнения кода
%matplotlib inline 
import numpy as np # для эффективной работы с вычислениями и большими массивами

**Выведем информацию на экран**

In [3]:
import pandas as pd
df=pd.read_csv('data.csv')
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


***Описание данных:***

* *children* - количество детей в семье;
* *days_employed* - общий трудовой стаж в семье;
* *dob_years* - возраст клиента;
* *education* - уровень образования клиента;
* *education_id* - идентификатор уровня образования;
* *family_status* - семейное положение;
* *family_status_id* - идентификатор семейного положения;
* *gender* - пол клиента;
* *income_type* - тип занятости;
* *debt* - имел ли задолженность по возврату кредитов;
* *total_income* - ежемесячный доход;
* *purpose* - цель получения кредита

**Изучим общую информацию по данным кредитного отдела банк**

* с помощью функции info() получим общую информацию о таблице;
* с помощью функции describe() получим общие статистические данные для каждого столбца таблицы.


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 1.6+ MB


* таблица с данными состоит из 21525 строк и 12 столбцов;
* в столбцах `days_employed` и `total_income` есть отсутствующие значения. Возможно, клиенты банка отказались предоставлять информацию или данные затерялись/некорректно отобразились. Так как нет возможности обратиться к инженеру проекта, буду считать, что клиенты отказались предоставлять информацию и заполню пропущенные ячейки самостоятельно, исходя из доступных данных таблицы;
* тип данных в столбцах `days_employed` и `total_income` - вещественный, подобная точность нам не нужна, заменим на целочисленный с помощью функции astype(), в остальных столбцах тип данных корректный;
* наименования столбцов соответсвуют их содержанию, воспользуемся функцией set_axis(), чтобы комфортно работать с данными без возможных ошибок. 


In [6]:
#исключим пропуски в наименовании столбцов, для этого переименуем их и сохраним данные в исх. таблице
df.set_axis(['children', 'days_employed', 'dob_years', 'education', 'education_id', 'family_status', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose'], axis='columns', inplace=True)
df.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

In [7]:
df.describe() 

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


* В столбце `children` min значение -1, в данные закралась ошибка, исправим ее, взяв значения этого столбца по модулю с помощью функции abs();
* В столбце `days_employed` есть отрицательные значения, в данные закралась ошибка, исправим ее, взяв значения этого столбца по модулю;
* В столбце `children` max значение 20 - это очень много, проверим, у какого количества клиентов такие данные с помощью функции count(), чтобы избежать ошибок;
* min значение в столбце dob_years = 0, скорее всего это ошибка, в реальной жизни можно обратиться к инженеру проекта за разъяснением, посмотрим, в каком количестве данных содержится ошибка с помощью функции count().


In [8]:
#заменим отрицательные значения в столбце days_employed и children на положительные
df['days_employed'] = df['days_employed'].abs()
df['children'] = df['children'].abs()
print(df.loc[df['children'] == 20]['children'].count())
print(df.loc[df['dob_years'] == 0]['dob_years'].count())

76
101


***Итоги:***
* исправили отрицательные значения в столбцах на положительные;
* подготовили названия столбцов для дальнейшей работы;
* обнаруженные ошибки в столбцах `dob_years` и `children` содержатся менее, чем в 1% от общего объема данных, соответственно, на итоговые результаты исследования не должны повлиять, поэтому оставим без изменений.

***Для дальнейшей работы:***
* необходимо обработать пропуски в данных;
* изменить тип данных в столбцах days_employed и total_income на целочисленный;
* привести значения столбца education к нижнему регистру, чтобы можно было в дальнейшем работать с данными.

<a id='1.1'></a>
### 1.1 Предобработка данных

**Обработка пропусков**

* посчитаем долю пропусков в столбцах с помощью функции mean();
* проверим, пропуски в столбцах в одних и тех же строках или в разных с помощью условия.

In [9]:
#считаем долю пропусков в столбцах
df.isnull().mean()

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

* В столбцах days_employed и total_income доля пропусков чуть больше 1%, это немного, но если их удалить, то потеряется информация из других столбцов, следовательно, целесообразнее будет их заполнить и после уже анализировать таблицу 

In [10]:
#проверяем, совпадают ли строки с пропусками
df[(df['days_employed'].isnull() == True) & (df['total_income'].isnull() == True)].count()

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64

* Да, строки с пропущенными данными совпадают в столбцах days_employed и total_income. Интересное наблюдение, можно предположить, что клиенты просто отказались эти данные предоставлять.

In [11]:
#Заполним пропущенные значения в столбцах характерными значениями, используя метод median(), для этого сгруппируем таблицу по столбцу income_type
m_table = df.groupby(['income_type']).agg({'total_income' : 'median'})
m_table


,total_income
income_type,
безработный,131339.751676
в декрете,53829.130729
госслужащий,150447.935283
компаньон,172357.950966
пенсионер,118514.486412
предприниматель,499163.144947
сотрудник,142594.396847
студент,98201.625314


In [12]:
df.loc[df['income_type'] == 'безработный', 'total_income'] = df.loc[df['income_type'] == 'безработный', 'total_income'].fillna(131339.751676)
df.loc[df['income_type'] == 'в декрете', 'total_income'] = df.loc[df['income_type'] == 'в декрете', 'total_income'].fillna(53829.130729)
df.loc[df['income_type'] == 'госслужащий', 'total_income'] = df.loc[df['income_type'] == 'госслужащий', 'total_income'].fillna(150447.935283)
df.loc[df['income_type'] == 'компаньон', 'total_income'] = df.loc[df['income_type'] == 'компаньон', 'total_income'].fillna(172357.950966)
df.loc[df['income_type'] == 'пенсионер', 'total_income'] = df.loc[df['income_type'] == 'пенсионер', 'total_income'].fillna(118514.486412)
df.loc[df['income_type'] == 'предприниматель', 'total_income'] = df.loc[df['income_type'] == 'предприниматель', 'total_income'].fillna(499163.144947)
df.loc[df['income_type'] == 'сотрудник', 'total_income'] = df.loc[df['income_type'] == 'сотрудник', 'total_income'].fillna(142594.396847)
df.loc[df['income_type'] == 'студент', 'total_income'] = df.loc[df['income_type'] == 'студент', 'total_income'].fillna(98201.625314)

In [13]:
#Заменим таким же образом пропущенные значения в столбце days_employed
m_table = df.groupby(['income_type']).agg({'days_employed' : 'median'})
m_table

,days_employed
income_type,
безработный,366413.652744
в декрете,3296.759962
госслужащий,2689.368353
компаньон,1547.382223
пенсионер,365213.306266
предприниматель,520.848083
сотрудник,1574.202821
студент,578.751554


In [14]:
df.loc[df['income_type'] == 'безработный', 'days_employed'] = df.loc[df['income_type'] == 'безработный', 'days_employed'].fillna(366413.652744)
df.loc[df['income_type'] == 'в декрете', 'days_employed'] = df.loc[df['income_type'] == 'в декрете', 'days_employed'].fillna(3296.759962)
df.loc[df['income_type'] == 'госслужащий', 'days_employed'] = df.loc[df['income_type'] == 'госслужащий', 'days_employed'].fillna(2689.368353)
df.loc[df['income_type'] == 'компаньон', 'days_employed'] = df.loc[df['income_type'] == 'компаньон', 'days_employed'].fillna(1547.382223)
df.loc[df['income_type'] == 'пенсионер', 'days_employed'] = df.loc[df['income_type'] == 'пенсионер', 'days_employed'].fillna(365213.306266)
df.loc[df['income_type'] == 'предприниматель', 'days_employed'] = df.loc[df['income_type'] == 'предприниматель', 'days_employed'].fillna(520.848083)
df.loc[df['income_type'] == 'сотрудник', 'days_employed'] = df.loc[df['income_type'] == 'сотрудник', 'days_employed'].fillna(1574.202821)
df.loc[df['income_type'] == 'студент', 'days_employed'] = df.loc[df['income_type'] == 'студент', 'days_employed'].fillna(578.751554)

In [15]:
#проверка исходной таблицы на пропущенные значения
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


***Итоги:***

Заменили пропущенные значения столбца total_income и  days_employed на медианные значения в зависимости от income_type, теперь все данные в таблице заполнены без пропусков и корректно.

**Замена типа данных**

* изменим тип данных в столбцах days_employed и total_income на целочисленный
* приведем значения столбца education к нижнему регистру, чтобы можно было в дальнейшем категоризировать клиентов по данному столбцу

In [16]:
#изменим тип столбцов total_income иdays_employed на int
df['total_income'] = df['total_income'].astype('int')
df['days_employed'] = df['days_employed'].astype('int')

#проверка
df.dtypes

children             int64
days_employed        int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
dtype: object

In [17]:
#приведем все значения столбца education в нижний регистр
df['education']=df['education'].str.lower()

#проверка
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

**Обработка дубликатов**

In [18]:
df.duplicated().sum()

71

* Так как в исходных данных нет id пользователя, будем считать, что данные и правда продублировались, для их удаления будем использовать метод drop_duplicates(), присвоим новые индексы для данных с помощью метода reset_index(drop=True)

In [19]:
df=df.drop_duplicates().reset_index(drop=True)

In [20]:
#проверка
df.duplicated().sum()

0

***Итоги***

В исходной таблице мы нашли 71 строку с дубликатами, скорее всего, это все данные одного человека, но так как в данных нет id клиента, узнать наверняка мы не можем, после обработки дубликатов в таблице остались только уникальные данные, которые можно использовать для анализа.


<a id='2'></a>
### 2. Лемматизация

* создадим словарь для целей кредита, столбец purpose;
* лемматизируем исходные цели и создадим соответствующие более краткие категории, добавим новые категории в словарь с помощью метода apply();
* добавим новый столбец в исходную табличку с данными по категориям целей кредита с помощью метода merge().

In [21]:
#Создаем словарик
slovarik = pd.DataFrame(data = df.purpose.unique(), columns = ['purpose'])
slovarik

,purpose
0,покупка жилья
1,приобретение автомобиля
2,дополнительное образование
3,сыграть свадьбу
4,операции с жильем
5,образование
6,на проведение свадьбы
7,покупка жилья для семьи
8,покупка недвижимости
9,покупка коммерческой недвижимости


* Нашли уникальные значения столбца с целями кредита и создали словарик

In [22]:
#лемматизируем исходные цели и создадим соответствующие более краткие категории, добавим новые категории в словарь
from pymystem3 import Mystem
m = Mystem()
 
def purpose_category(slovarik):
   
    lemmas = m.lemmatize(slovarik)
    
    if ('недвижимость' in lemmas) or ('жилье' in lemmas):
        return 'недвижимость'
 
    elif 'образование' in lemmas:
        return 'образование'
 
    elif 'автомобиль' in lemmas:
        return 'автомобиль'
 
    elif 'свадьба' in lemmas:
        return 'свадьба'
    
slovarik['category'] = slovarik['purpose'].apply(purpose_category)
slovarik

,purpose,category
0,покупка жилья,недвижимость
1,приобретение автомобиля,автомобиль
2,дополнительное образование,образование
3,сыграть свадьбу,свадьба
4,операции с жильем,недвижимость
5,образование,образование
6,на проведение свадьбы,свадьба
7,покупка жилья для семьи,недвижимость
8,покупка недвижимости,недвижимость
9,покупка коммерческой недвижимости,недвижимость


* Получили словарик с более краткими категориями целей кредита

In [23]:
#теперь склеим исходную табличку и полученный словарь
df=df.merge(slovarik, on='purpose', how='left')
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,недвижимость
21450,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,автомобиль
21451,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость
21452,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль


***Итоги***

Теперь мы можем категоризировать данные по целям кредита и определить его влияние на возврат кредита в срок.


<a id='3'></a>
### 3. Категоризация данных

Проведем категоризацию данных:

* по ежемесячному доходу, столбец `'total_income'`
* по количеству детей, столбец `'children'`
* по количеству отработанных дней, столбец `'days_employed'` - 
* по возрасту, столбец `'dob_years'`

In [64]:
#медианное, минимальное и максимальное значения столбца total_income:
print(df['total_income'].min())
print(df['total_income'].max())
print(df['total_income'].median())

20667
2265604
142594.0


Категоризируем столбец `total_income`, считая, что:
 * клиенты, получающие до 50000 имеют низкий доход
 * клиенты, получающие до 50000 - 150000 имеют средний доход
 * клиенты, получающие больше 150000 имеют высокий доход

In [65]:
def income_group(row):
    if row['total_income'] <= 50000:
        return 'низкий доход'
    elif 50000 < row['total_income'] <= 150000:
        return 'средний доход'
    else:
        return 'высокий доход'

df['income_group'] = df.apply(income_group, axis=1)
df.groupby('total_income_group')['total_income'].count()

total_income_group
(0.0, 50000.0]           372
(50000.0, 100000.0]     4091
(100000.0, inf]        16991
Name: total_income, dtype: int64

In [66]:
def children_group(row):
    if row['children'] < 1:
        return 'нет детей'
    elif 1 < row['children'] <= 2:
        return '1-2 ребенка'
    else:
        return 'более 2 детей'

df['children_group'] = df.apply(children_group, axis=1)
df.groupby('children_group')['children'].count()

children_group
1-2 ребенка       2052
более 2 детей     5311
нет детей        14091
Name: children, dtype: int64

In [67]:
def days_employed_group(row):
    if row['days_employed'] < 3650:
        return 'до 10 лет'
    elif 3650 < row['days_employed'] <= 12775:
        return 'от 10 до 35 лет'
    else:
        return 'более 35 лет'

df['days_employed_group'] = df.apply(days_employed_group, axis=1)
df.groupby('days_employed_group')['days_employed'].count()

days_employed_group
более 35 лет        3893
до 10 лет          14474
от 10 до 35 лет     3087
Name: days_employed, dtype: int64

In [68]:
def dob_years_group(row):
    if row['dob_years'] < 35:
        return 'молодой'
    elif 35 < row['dob_years'] <= 65:
        return 'средний возраст'
    else:
        return 'пожилой'

df['dob_years_group'] = df.apply(dob_years_group, axis=1)
df.groupby('dob_years_group')['dob_years'].count()

dob_years_group
молодой             6068
пожилой             1318
средний возраст    14068
Name: dob_years, dtype: int64

In [69]:
#Выведем общую таблицу с категоризированными данными
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,category,total_income_group,income_group,children_group,days_employed_group,dob_years_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,"(100000.0, inf]",высокий доход,более 2 детей,от 10 до 35 лет,средний возраст
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,"(100000.0, inf]",средний доход,более 2 детей,от 10 до 35 лет,средний возраст
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,"(100000.0, inf]",средний доход,нет детей,от 10 до 35 лет,молодой
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,"(100000.0, inf]",высокий доход,более 2 детей,от 10 до 35 лет,молодой
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,"(100000.0, inf]",высокий доход,нет детей,более 35 лет,средний возраст


<a id='4'></a>
### 4. Анализ данных

<a id='4.1'></a>
**Определим зависимость между наличием детей и возвратом кредита в срок**

In [54]:
df.pivot_table(index='children_group',values='debt',aggfunc=['mean','count'])

,mean,count
,debt,debt
children_group,,
1-2 ребенка,0.094542,2052
более 2 детей,0.091132,5311
нет детей,0.075438,14091


* В семьях с детьми вероятность просрочки выше

<a id='4.2'></a>
**Определим зависимость между семейным положением и возвратом кредита в срок**

In [55]:
df.pivot_table(index='family_status',values='debt',aggfunc=['mean','count'])

,mean,count
,debt,debt
family_status,,
Не женат / не замужем,0.097509,2810
в разводе,0.071130,1195
вдовец / вдова,0.065693,959
гражданский брак,0.093471,4151
женат / замужем,0.075452,12339


* У тех клиентов, кто был желан/замужем или находится в данном статусе, вероятность просрочки меньше, чем у тех, кто официально не узаконил свои отношения.

<a id='4.3'></a>
**Определим зависимость между уровнем дохода и возвратом кредита в срок?**

In [56]:
df.pivot_table(index='income_group',values='debt',aggfunc=['mean','count'])

,mean,count
,debt,debt
income_group,,
высокий доход,0.077612,9831
низкий доход,0.061828,372
средний доход,0.084881,11251


* Можно было бы предположить, что у людей с большим количеством дохода вероятность просрочки должна быть самая низкая, но результаты исследования показывают, что наименьшая вероятность просрочки у клиентов с низким доходом.

<a id='4.4'></a>
**Зависимость цели кредита и его возвратом в срок**

In [57]:
df.pivot_table(index='category',values='debt',aggfunc=['mean','count'])

,mean,count
,debt,debt
category,,
автомобиль,0.093590,4306
недвижимость,0.072334,10811
образование,0.092200,4013
свадьба,0.080034,2324


 * Выдача кредита на категории "свадьба" и "недвижимость" имеют меньшьшую вероятность просрочки по срокам, нежели категории "образование" и "автомобиль". Возможно, такие результаты исследования подтверждают исследования зависимости возврата кредита в срок и семейного положения. 

<a id='5'></a>
## Общий вывод:

Отвечая на вопрос задания "влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок" - да, клиенты в браке и без детей - самые благоприятные заемщики.

По результатам проведенного исследования можно сделать вывод, что:
* самые надежные креднитные заемщики - это клиенты с узаконенныи отношениями, без детей, с целью кредита - недвижимость или свадьба. А вот высокий уровень дохода не повышает вероятность возврата кредита в срок.
* неблагоприятные заемщики - это клиенты банка, которые не находятся в браке, с несколькими детьми, берущие кредит на автомобиль или образование.
